In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score,confusion_matrix,average_precision_score
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        


# Any results you write to the current directory are saved as output.

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

/kaggle/input/feature/end_data.csv
/kaggle/input/wns-analyticswizard/sample_submission_ipsblct/sample_submission.csv
/kaggle/input/wns-analyticswizard/test_aq1fgdb/test.csv
/kaggle/input/wns-analyticswizard/train_na17sgz/item_data.csv
/kaggle/input/wns-analyticswizard/train_na17sgz/train.csv
/kaggle/input/wns-analyticswizard/train_na17sgz/view_log.csv
/kaggle/input/nearest-date/Nearest_date.csv


/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [2]:
train= pd.read_csv("/kaggle/input/wns-analyticswizard/train_na17sgz/train.csv")
item_data=pd.read_csv("/kaggle/input/wns-analyticswizard/train_na17sgz/item_data.csv")
view_log=pd.read_csv("/kaggle/input/wns-analyticswizard/train_na17sgz/view_log.csv")
test = pd.read_csv("/kaggle/input/wns-analyticswizard/test_aq1fgdb/test.csv")
sample = pd.read_csv("/kaggle/input/wns-analyticswizard/sample_submission_ipsblct/sample_submission.csv")
train['impression_time']=pd.to_datetime(train['impression_time'])
test['impression_time']=pd.to_datetime(test['impression_time'])
view_log['server_time']=pd.to_datetime(view_log['server_time'])
test['is_click']=np.nan
train['data']='train'
test['data']='test'
test=test[train.columns]
cd_all=pd.concat([train,test],axis=0)

In [3]:
print(train.dtypes)
print("-----------------------")
print(train.shape)
print(item_data.shape)
print(view_log.shape)
print(test.shape)

impression_id              object
impression_time    datetime64[ns]
user_id                     int64
app_code                    int64
os_version                 object
is_4G                       int64
is_click                    int64
data                       object
dtype: object
-----------------------
(237609, 8)
(132761, 6)
(3118622, 5)
(90675, 8)


In [4]:
train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,data
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0,train
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1,train
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0,train
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0,train
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0,train


In [5]:
#Create new data d1 like this : Sort cd_all by user_id and impression date [ remove other columns ]
#Remove duplicate observations from d1 
d1=cd_all.sort_values(['user_id','impression_time'])

d1=d1[['user_id','impression_time']]

print(d1.shape)
d1=d1.drop_duplicates()
print(d1.shape)

(328284, 2)
(327442, 2)


In [6]:
#Create new data d2 : sort cd_viewlog by user_id and server_date [ remove other columns ]
d2=view_log.sort_values(['user_id','server_time'])
d2=d2[['user_id','server_time']]
print(d2.shape)
d2=d2.drop_duplicates()
print(d2.shape)

(3118622, 2)
(2712976, 2)


In [7]:
#end_data={'user_id':[],'impression_time':[],'result':[]}

#for id in tqdm_notebook(d1['user_id'].unique()):
#    for date in d1.loc[d1['user_id']==id,'impression_time']:
#        d2_sub=d2[d2['user_id']==id]
#        result=(pd.to_datetime(date-d2_sub['server_time']).dt.day<10).sum()
#        end_data['user_id'].append(id)
#        end_data['impression_time'].append(date)
#        end_data['result'].append(result)

#The above for loop  generate data in end_date, i saved that csv file and loading it to save the execution time
end_data= pd.read_csv("/kaggle/input/feature/end_data.csv")


#end_data=pd.DataFrame(end_data)

In [8]:
print(end_data.shape)
print(train.shape)
print(item_data.shape)
print(view_log.shape)

(327442, 3)
(237609, 8)
(132761, 6)
(3118622, 5)


In [9]:
end_data.head()

,user_id,impression_time,result
0,0,2018-11-26 23:30:00,37
1,1,2018-12-13 19:49:00,7
2,2,2018-11-19 20:49:00,139
3,2,2018-11-20 20:29:00,126
4,2,2018-11-20 20:53:00,126


In [10]:
#Nearest_date={'user_id':[],'impression_time':[],'Nearest_serverdate':[]}
#for id in tqdm_notebook(d1['user_id'].unique()):   #tqdm_notebook will give display graphical progress bar
#    for date in d1.loc[d1['user_id']==id,'impression_time']:
#        d2_sub=d2[d2['user_id']==id]
#        b=51909
#        date2 = pd.to_datetime('2018-11-26 23:30:00')
#        for date1 in d2_sub['server_time']:
#            a=pd.to_datetime(date)-pd.to_datetime(date1)
#            k=a/pd.Timedelta(1, unit='d')
#            if k>0:        
#                if k>=b:
#                    b=b
#                    date2=date2
#                else:
#                    b=k
#                    date2=date1
#        Nearest_date['user_id'].append(id)
#        Nearest_date['impression_time'].append(date)
#        Nearest_date['Nearest_serverdate'].append(date2)


#pd.DataFrame(Nearest_date).to_csv("Nearest_date.csv",index=False)
Nearest_date= pd.read_csv("../input/nearest-date/Nearest_date.csv")

In [11]:
Nearest_date.head()

,user_id,impression_time,Nearest_serverdate
0,0,2018-11-26 23:30:00,2018-11-26 15:34:00
1,1,2018-12-13 19:49:00,2018-12-09 01:37:00
2,2,2018-11-19 20:49:00,2018-11-19 14:50:00
3,2,2018-11-20 20:29:00,2018-11-19 21:46:00
4,2,2018-11-20 20:53:00,2018-11-19 21:46:00


In [12]:
print(cd_all.dtypes)
print("===========")
print(end_data.dtypes)
print("===========")
print(Nearest_date.dtypes)

impression_id              object
impression_time    datetime64[ns]
user_id                     int64
app_code                    int64
os_version                 object
is_4G                       int64
is_click                  float64
data                       object
dtype: object
user_id             int64
impression_time    object
result              int64
dtype: object
user_id                int64
impression_time       object
Nearest_serverdate    object
dtype: object


In [13]:
end_data['impression_time']=pd.to_datetime(end_data['impression_time']) # changing the dtype
Nearest_date['impression_time']=pd.to_datetime(Nearest_date['impression_time']) # changing the dtype
Nearest_date['Nearest_serverdate']=pd.to_datetime(Nearest_date['Nearest_serverdate']) # changing the dtype

In [14]:
cd_all=pd.merge(cd_all,end_data,on=['user_id','impression_time'],how='left')

cd_all=pd.merge(cd_all,Nearest_date,on=['user_id','impression_time'],how='left')

#cd_all=pd.merge(cd_all,view_log,how='left',left_on=['user_id','Nearest_serverdate'],right_on=['user_id','server_time'])
#cd_all=pd.merge(cd_all,item_data,on='item_id',how='left')

In [15]:
cd_all.shape

(328284, 10)

In [16]:
cd_all.dtypes

impression_id                 object
impression_time       datetime64[ns]
user_id                        int64
app_code                       int64
os_version                    object
is_4G                          int64
is_click                     float64
data                          object
result                         int64
Nearest_serverdate    datetime64[ns]
dtype: object

In [17]:
from math import ceil

def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

In [18]:
cd_all["imp_month"] = cd_all['impression_time'].dt.month
cd_all['imp_wom']=cd_all['impression_time'].apply(lambda x: week_of_month(x))
cd_all["imp_weekday"] = cd_all['impression_time'].dt.weekday

In [19]:
cd_all["near_month"] = cd_all['Nearest_serverdate'].dt.month
cd_all['near_wom']=cd_all['Nearest_serverdate'].apply(lambda x: week_of_month(x))
cd_all["near_weekday"] = cd_all['Nearest_serverdate'].dt.weekday

In [20]:
# dropping  column
for col in ['impression_time','Nearest_serverdate']:
    cd_all.drop([col],axis=1,inplace=True)
  

In [21]:
cd_all['os_version']=cd_all['os_version'].map({'latest':3,'intermediate':2,'old':'1'})
cd_all['os_version']=cd_all['os_version'].astype(int)

In [22]:
cd_all.dtypes

impression_id     object
user_id            int64
app_code           int64
os_version         int64
is_4G              int64
is_click         float64
data              object
result             int64
imp_month          int64
imp_wom            int64
imp_weekday        int64
near_month         int64
near_wom           int64
near_weekday       int64
dtype: object

In [23]:
temp=pd.get_dummies(cd_all['imp_month'],drop_first=True)
cd_all=pd.concat([cd_all,temp],axis=1)
cd_all.drop(['imp_month'],axis=1,inplace=True)

temp=pd.get_dummies(cd_all['imp_wom'],drop_first=True)
cd_all=pd.concat([cd_all,temp],axis=1)
cd_all.drop(['imp_wom'],axis=1,inplace=True)

temp=pd.get_dummies(cd_all['imp_weekday'],drop_first=True)
cd_all=pd.concat([cd_all,temp],axis=1)
cd_all.drop(['imp_weekday'],axis=1,inplace=True)

temp=pd.get_dummies(cd_all['near_month'],drop_first=True)
cd_all=pd.concat([cd_all,temp],axis=1)
cd_all.drop(['near_month'],axis=1,inplace=True)

temp=pd.get_dummies(cd_all['near_wom'],drop_first=True)
cd_all=pd.concat([cd_all,temp],axis=1)
cd_all.drop(['near_wom'],axis=1,inplace=True)

temp=pd.get_dummies(cd_all['near_weekday'],drop_first=True)
cd_all=pd.concat([cd_all,temp],axis=1)
cd_all.drop(['near_weekday'],axis=1,inplace=True)


temp=pd.get_dummies(cd_all['os_version'],drop_first=True)
cd_all=pd.concat([cd_all,temp],axis=1)
cd_all.drop(['os_version'],axis=1,inplace=True)

In [24]:
cd_all['app_code']=cd_all['app_code'].astype(str)
for col in ['app_code']:
    freqs=cd_all[col].value_counts()
    k=freqs.index[freqs>=100][:-1]
    for cat in k:
        name=col+'_'+cat
        cd_all[name]=(cd_all[col]==cat).astype(int)
    del cd_all[col]
    print(col)

app_code


In [25]:
cd_all=cd_all.rename(columns={'result': 'count_in_log_for10days'})

In [26]:
cd_all.drop(['user_id'],axis=1,inplace=True)  

In [27]:
cd_all.shape

(328284, 162)

In [28]:
train_df=cd_all[cd_all['data']=='train']
del train_df['data']
test_df=cd_all[cd_all['data']=='test']
test_df.drop(['is_click','data'],axis=1,inplace=True)

In [29]:
print(train_df.shape)
print(test_df.shape)

(237609, 161)
(90675, 160)


In [30]:
train_df.drop(['impression_id'],axis=1,inplace=True)
test_df.drop(['impression_id'],axis=1,inplace=True)

In [31]:
#t1,t2 = train_test_split(train_df,test_size=0.30,stratify=train['is_click'],random_state=12)

#t1_x=t1.drop(['is_click'],axis=1)
#t1_y=t1['is_click']
#t2_x=t2.drop(['is_click'],axis=1)
#t2_y=t2['is_click']

#t1col=t1_x.columns
#t2col=t2_x.columns

#ros = RandomOverSampler(random_state=143)
#t1_x,t1_y = ros.fit_resample(t1_x,t1_y)

#scaler=StandardScaler()
#scaler.fit(t1_x)
#t1_x=scaler.transform(t1_x)
#scaler.fit(t2_x)
#t2_x=scaler.transform(t2_x)

#t1_x = pd.DataFrame(t1_x, columns = t1col)
#t2_x=pd.DataFrame(t2_x, columns = t2col)

In [32]:
train_df_x=train_df.drop(['is_click'],axis=1)
train_df_y=train_df['is_click']
test_df_x=test_df

train_dfcol=train_df_x.columns
test_dfcol=test_df_x.columns

ros = RandomOverSampler(random_state=143)
train_df_x,train_df_y = ros.fit_resample(train_df_x,train_df_y)

scaler=StandardScaler()
scaler.fit(train_df_x)
train_df_x=scaler.transform(train_df_x)
scaler.fit(test_df_x)
test_df_x=scaler.transform(test_df_x)

train_df_x = pd.DataFrame(train_df_x, columns = train_dfcol)
test_df_x=pd.DataFrame(test_df_x, columns = test_dfcol)

In [33]:
# params={'class_weight':['balanced'],
#        'penalty':['l2'],
#         'C':[0.05]
#       }

#glm=LogisticRegression(fit_intercept=True)
#grid_search=GridSearchCV(glm,cv=3,param_grid=params,scoring="roc_auc",verbose=True,n_jobs=-1)
#grid_search.fit(train_df_x,train_df_y)
#grid_search.best_estimator_
lgb_model = LGBMClassifier()
lgb_model.fit(train_df_x, train_df_y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [34]:
test_predict=lgb_model.predict_proba(test_df_x)
is_click=pd.DataFrame(test_predict[:,1])

In [35]:
output=pd.concat([test['impression_id'],is_click],axis=1)
output=output.rename(columns={0: 'is_click'})

In [36]:
pd.DataFrame(output).to_csv("output.csv",index=False)

In [37]:
#real=t2_y
#for cutoff in np.linspace(0,1,11):
#    predicted=(test_predict[:,1]>cutoff).astype(int)
#    print(round(cutoff,1),roc_auc_score(real, predicted))   

In [38]:
#real=t2_y
#roc_auc_score(real,grid_search.predict_proba(t2_x)[:,1])

In [39]:
sample

,impression_id,is_click
0,a9e7126a585a69a32bc7414e9d0c0ada,0
1,caac14a5bf2ba283db7708bb34855760,0
2,13f10ba306a19ce7bec2f3cae507b698,0
3,39c4b4dc0e9701b55a0a4f072008fb3f,0
4,bf5a572cca75f5fc67f4b14e58b11d70,0
5,ca09514449e304fee91456fb0918667a,0
6,b8b35cb0ae1719aa9f0ba7a9f3785687,0
7,33502d1e85818b9b3414c8c006c34e07,0
8,5314e09fb680c56b58eb5a3b5b7bce0c,0
9,e1a8a26385f49a5a7e04d34a5823bff3,0
